In [1]:
import numpy as np
import pandas as pd
import logging
import json
import sys
import selenium
import urllib.parse as urlparse
from operator import itemgetter
import time
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
import requests
from bs4 import BeautifulSoup
from flask import Flask, render_template, request, redirect, url_for, abort
import requests
import copy
import config
import datetime
from IPython.display import display, HTML


# Create the Flask application.
app = Flask(__name__)
app.logger.setLevel(logging.INFO)
app.config.from_object(config.ProductionConfig)
from selenium.webdriver import Chrome
from webdriver_manager.chrome import ChromeDriverManager

import time


In [2]:
def call_espn_api(league_id):
    app.logger.info('%s - Calling ESPN API', league_id)
    url = ("https://fantasy.espn.com/apis/v3/games/fba/seasons/2022/segments/0/leagues/{}?&view=mMatchupScore&view=mPendingTransactions&view=mPositionalRatings&view=mRoster&view=mTeam&view=modular)".format(league_id))
    try:
        r = requests.get(url)
    except requests.exceptions.RequestException as ex:
        app.logger.error('%s - Could not make ESPN API call', league_id, ex)
        abort(500)
        sys.exit('Could not make ESPN API call.')
    data = r.json()
    print(r)
    return data

In [3]:
class Player:
    def __init__(self, name):
        self.name = name
        self.fga = 1.0
        self.fgm = 0
        self.fgp = self.fgm / self.fga
        self.fta = 1.0
        self.ftm = 0
        self.ftp = self.ftm / self.fta
        self.pts = 0
        self.ast = 0
        self.rbs = 0
        self.tov = 0
        self.m3s = 0
        self.stl = 0
        self.blk = 0
    def __str__(self):
        return self.name + " pts/reb/ast: " + format(self.pts ,'.2f') + "/" + format(self.rbs ,'.2f') + "/" + format(self.ast ,'.2f') + " stl/blk/tov: " + format(self.stl ,'.2f') + "/" + format(self.blk ,'.2f') + "/" + format(self.tov ,'.2f') + "/" + " fg%/ft%/3pm: " + format(self.fgp ,'.2f') + "/" + format(self.ftp ,'.2f') + "/" + format(self.m3s ,'.2f')

In [4]:
def parse_api_to_rosters(hashtag, statPeriod=0):
    data = call_espn_api(1807480577)
    rosters = [[Player("") for i in range(14)] for j in range(12)]
    teamMap = {}
    df = pd.read_csv("hashtag2021.csv")
    df["PLAYER"] = [x.rstrip() for x in df["PLAYER"]]
    ir_players = ["Kawhi Leonard", "Klay Thompson",
                 "Zion Williamson", "Pascal Siakam", "Brook Lopez","Victor Oladipo","Kyrie Irving", "Bobby Portis"]
#     ir_players = ["LeBron James", "Shai Gilgeous-Alexander", "Terrence Ross", "Aaron Gordon", "Andre Drummond", "Evan Fournier", "Kevin Love", "Mitchell Robinson"]
    for i in range(12):
        count = 0
        print("\n")
        teamMap[data["teams"][i]['abbrev']] = i
        for j in range(len(data["teams"][i]['roster']['entries'])):
            rosters[i][j] = Player(data["teams"][i]['roster']['entries'][j]['playerPoolEntry']['player']['fullName'])
            if rosters[i][j].name in ir_players:
                continue
            count += 1
            if rosters[i][j].name == "Robert Williams III":
                rosters[i][j].name = "Robert Williams"
            if rosters[i][j].name == "P.J. Washington":
                rosters[i][j].name = "PJ Washington"
            if (hashtag and rosters[i][j].name in df.values):
                row = df.loc[df["PLAYER"] == rosters[i][j].name]
                print(rosters[i][j].name + " (hashtag)") 
                
                rosters[i][j].pts = float(row["PTS"].values[0])
                rosters[i][j].rbs = float(row["TREB"].values[0])
                rosters[i][j].ast = float(row["AST"].values[0])
                rosters[i][j].stl = float(row["STL"].values[0])
                rosters[i][j].blk = float(row["BLK"].values[0])
                rosters[i][j].m3s = float(row["3PM"].values[0])
                rosters[i][j].tov = float(row["TO"].values[0])
                
                rosters[i][j].fgm = float(row["FG%"].values[0].split("/")[0].split("(")[1])
                rosters[i][j].fga = float(row["FG%"].values[0].split("/")[1][:-1])
                rosters[i][j].fgp = rosters[i][j].fgm / rosters[i][j].fga

                rosters[i][j].ftm = float(row["FT%"].values[0].split("/")[0].split("(")[1])
                rosters[i][j].fta = float(row["FT%"].values[0].split("/")[1][:-1])
                rosters[i][j].ftp = rosters[i][j].ftm / rosters[i][j].fta
            else:
                print(rosters[i][j].name + " (espn)")
                x = data["teams"][i]['roster']['entries'][j]['playerPoolEntry']['player']['stats']
                statIndex = 0
                for k in range(len(x)):
                    if x[k]['seasonId'] == 2021 and x[k]['statSplitTypeId'] == statPeriod and x[k]['externalId'] == '2021' and x[k]['statSourceId'] == 0:
                        statIndex = k
                        if not x[k]['stats'] or x[k]['averageStats']['0'] == 0:
                            for l in range(len(x)):
                                if x[l]['seasonId'] == 2021 and x[l]['statSplitTypeId'] == 0 and x[l]['externalId'] == '2021' and x[l]['statSourceId'] == 1:
                                    statIndex = l
#                                     if x[l]['averageStats']['0'] == 0:
#                                         for m in range(len(x)):
#                                             if x[m]['seasonId'] == 2020 and x[l]['statSplitTypeId'] == 0 and x[l]['externalId'] == '2020' and x[l]['statSourceId'] == 1:
#                                                 statIndex = m
                
                seasonStats = x[statIndex]['averageStats']

                rosters[i][j].pts = seasonStats['0']
                rosters[i][j].fgm = seasonStats['13']
                rosters[i][j].fga = seasonStats['14']
                if (seasonStats['14'] == 0):
                    rosters[i][j].fgp = 0
                else:
                    rosters[i][j].fgp = seasonStats['13'] / seasonStats['14']
                rosters[i][j].ftm = seasonStats['15']
                rosters[i][j].fta = seasonStats['16']
                if (seasonStats['16'] == 0):
                    rosters[i][j].ftp = 0
                else:
                    rosters[i][j].ftp = seasonStats['15'] / seasonStats['16']
                rosters[i][j].rbs = seasonStats['6']
                rosters[i][j].ast = seasonStats['3']
                rosters[i][j].stl = seasonStats['2']
                rosters[i][j].blk = seasonStats['1']
                rosters[i][j].tov = seasonStats['11']
                if '17' not in seasonStats:
                    rosters[i][j].m3s = 0
                else:
                    rosters[i][j].m3s = seasonStats['17']
        print(count)
    return rosters, teamMap

In [5]:
def addTeamData(rosters, category):
    team_data = [0 for j in range(12)]
    for i in range(0, 12):
        val = 0
        for j in range(0, len(rosters[i])):
            val += getattr(rosters[i][j], category)
        team_data[i] = val
    return team_data
def rank_list(input, order):
    output = [0] * len(input)
    for i, x in enumerate(sorted(range(len(input)), key=lambda y: order *input[y])):
        output[x] = i + 1
    return output
def printRow(df, index):
    df = {x: y for x, y in zip(df.columns, df.loc[index])}
    sf = pd.DataFrame(df, index=[index])
    display(sf)
def trade_analyzer(rosters, team1, players_from1, team2, players_from2, teamMap):
    if len(players_from1) != len(players_from2):
        print("The number of players exchanged must be equal")
    for player in players_from1:
        [print(p) for p in rosters[teamMap[team1]] if p.name == player]
    for player in players_from2:
        [print(p) for p in rosters[teamMap[team2]] if p.name == player]
    old_data = genRankTable(rosters,teamMap)
    old_stats = genStatsTable(rosters,teamMap)
    old_rankList, matchups = genRankList(old_data)
    print("Prior to the trade " + team1 + " was ranked "+ str(old_rankList[teamMap[team1]]))
    printRow(old_data, team1)
    printRow(old_stats, team1)
    print("Prior to the trade " + team2 + " was ranked "+ str(old_rankList[teamMap[team2]]))
    printRow(old_data, team2)
    printRow(old_stats, team2)
    new_team1 = [player for player in rosters[teamMap[team1]] if player.name not in players_from1]
    new_team1 += [player for player in rosters[teamMap[team2]] if player.name in players_from2]
    new_team2 = [player for player in rosters[teamMap[team2]] if player.name not in players_from2]
    new_team2 += [player for player in rosters[teamMap[team1]] if player.name in players_from1]
    new_rosters =[[j for j in i] for i in rosters]
    new_rosters[teamMap[team1]] = new_team1
    new_rosters[teamMap[team2]] = new_team2
#     print([print(x) for x in new_rosters[teamMap[team1]]])
    new_data = genRankTable(new_rosters, teamMap)
    new_stats = genStatsTable(new_rosters, teamMap)
    new_rankList, matchups = genRankList(new_data)
    print("After the trade " + team1 + " was ranked "+ str(new_rankList[teamMap[team1]]))
    printRow(new_data, team1)
    printRow(new_stats, team1)
    print("After the trade " + team2 + " was ranked "+ str(new_rankList[teamMap[team2]]))
    printRow(new_data, team2)
    printRow(new_stats, team2)
def calculate_matchups(team_name, rank_table):
    my_team = rank_table.loc[team_name]
    would_beat = []
    would_lose = []
    overall_margin = []
    for i in range(12):
        margin = []
        score = 0
        curr_team = rank_table.iloc[i]
        if curr_team.name == my_team.name:
            continue
        for j in range(9):
            if my_team.iloc[j] < curr_team.iloc[j]:
                score += 1
                margin.append(curr_team.iloc[j] - my_team.iloc[j])
        if score >= 5:
            would_beat.append(curr_team.name)
        else:
            would_lose.append(curr_team.name)
        if margin == []:
            avg = 0
        else:
            avg = sum(margin)/len(margin)
#         print(my_team.name + " " + str(score) + " " + curr_team.name + " " + str(9 - score) + " by an average of " + str(avg))
    return would_beat, would_lose
def genRankList(table):
    num_beaten = [0 for i in range(0, table.shape[0])]
    teams = table.index
    matchups = [[0 for i in range(2)]for i in range(0, table.shape[0])]
    for i in range(len(teams)):
        beats, loses = calculate_matchups(teams[i],table)
        num_beaten[i] = len(beats)
        beats_str = " "
        for j in range(len(beats)):
            beats_str += beats[j] + ", "
        loses_str = " "
        for j in range(len(loses)):
            loses_str += loses[j] + ", "
        print(teams[i] + " beats" + beats_str + "and loses to" + loses_str)
        print("")
        matchups[i][0] = beats
        matchups[i][1] = loses
    return rank_list(num_beaten, -1), matchups
def genRankTable(rosters, teamMap):
    fga = addTeamData(rosters, "fga")
    fgm = addTeamData(rosters, "fgm")
    fg = [rank_list([fgm[i]/fga[i] for i in range(0, len(fga))], -1)]
    fta = addTeamData(rosters, "fta")
    ftm = addTeamData(rosters, "ftm")
    ft = [rank_list([ftm[i]/fta[i] for i in range(0, len(fta))], -1)]
    points = [rank_list(addTeamData(rosters, "pts"), -1)]
    rebounds = [rank_list(addTeamData(rosters, "rbs"), -1)]
    assists = [rank_list(addTeamData(rosters, "ast"), -1)]
    steals = [rank_list(addTeamData(rosters, "stl"), -1)]
    blocks = [rank_list(addTeamData(rosters, "blk"), -1)]
    turnovers = [rank_list(addTeamData(rosters, "tov"), 1)]
    threes = [rank_list(addTeamData(rosters, "m3s"), -1)]
    # total = [rankList(addTeamDataHashtag(hashtag_table, 'TOTAL', all_teams, name_mapping), -1)]
    data = [*zip(*(fg + ft + points + rebounds + assists + steals + blocks + turnovers + threes))]
    rank_table = pd.DataFrame(data, index = teamMap.keys(), columns = ['fg%', 'ft%', 'points','rebounds','assists','steals','blocks','turnovers','threes'])
    rank_table["Total"] = rank_table.sum(axis = 1, skipna = True)
    return rank_table
def genStatsTable(rosters, teamMap):
    fga = addTeamData(rosters, "fga")
    fgm = addTeamData(rosters, "fgm")
    fg = [[fgm[i]/fga[i] for i in range(0, len(fga))]]
    fta = addTeamData(rosters, "fta")
    ftm = addTeamData(rosters, "ftm")
    ft = [[ftm[i]/fta[i] for i in range(0, len(fta))]]
    points = [addTeamData(rosters, "pts")]
    rebounds = [addTeamData(rosters, "rbs")]
    assists = [addTeamData(rosters, "ast")]
    steals = [addTeamData(rosters, "stl")]
    blocks = [addTeamData(rosters, "blk")]
    turnovers = [addTeamData(rosters, "tov")]
    threes = [addTeamData(rosters, "m3s")]
    # total = [rankList(addTeamDataHashtag(hashtag_table, 'TOTAL', all_teams, name_mapping), -1)]
    data = [*zip(*(fg + ft + points + rebounds + assists + steals + blocks + turnovers + threes))]
    stats_table = pd.DataFrame(data, index = teamMap.keys(), columns = ['fg%', 'ft%', 'points','rebounds','assists','steals','blocks','turnovers','threes'])
    return stats_table


In [6]:
rosters, teamMap = parse_api_to_rosters(hashtag=True, statPeriod=1)
data = genRankTable(rosters, teamMap)
stats = genStatsTable(rosters, teamMap)
rankList, matchups = genRankList(data)
data["rank"] = rankList
display(data)
display(stats.round(2))
# stats["np.mean(stats["steals"])

[2021-10-25 15:57:49,210] INFO in <ipython-input-2-85be66810220>: 1807480577 - Calling ESPN API


<Response [200]>


Nikola Jokic (hashtag)
Shai Gilgeous-Alexander (hashtag)
LaMelo Ball (hashtag)
Jaren Jackson Jr. (hashtag)
OG Anunoby (hashtag)
Mikal Bridges (hashtag)
Jarrett Allen (hashtag)
Keldon Johnson (hashtag)
Marcus Smart (hashtag)
Kelly Olynyk (hashtag)
Tyler Herro (hashtag)
Nic Claxton (espn)
Will Barton (hashtag)
13


Giannis Antetokounmpo (hashtag)
Anthony Davis (hashtag)
Clint Capela (hashtag)
Ben Simmons (hashtag)
Lonzo Ball (hashtag)
Malcolm Brogdon (hashtag)
Spencer Dinwiddie (hashtag)
Scottie Barnes (hashtag)
Luguentz Dort (hashtag)
Terance Mann (hashtag)
Alperen Sengun (hashtag)
Nemanja Bjelica (hashtag)
Jae'Sean Tate (hashtag)
13


Karl-Anthony Towns (hashtag)
LeBron James (hashtag)
Russell Westbrook (hashtag)
Ja Morant (hashtag)
Robert Williams (hashtag)
Jalen Green (hashtag)
Kemba Walker (hashtag)
Ivica Zubac (hashtag)
Jordan Clarkson (hashtag)
Andre Drummond (hashtag)
Aaron Gordon (hashtag)
Ricky Rubio (hashtag)
Cole Anthony (hashtag)
13


Kevin Durant (hashtag

,fg%,ft%,points,rebounds,assists,steals,blocks,turnovers,threes,Total,rank
TANG,2,8,5,3,6,2,3,5,8,42,2
MAND,1,12,9,1,5,4,2,8,12,54,3
PATE,6,10,8,4,1,7,10,12,9,67,9
SWAN,10,7,4,10,7,6,5,6,2,57,8
GPp,5,3,1,11,10,9,6,4,6,55,6
VEGE,8,5,6,9,2,10,4,11,3,58,7
SHAR,9,4,3,6,11,11,8,2,1,55,5
GREW,4,2,7,2,12,3,1,3,7,41,1
RD,3,6,10,5,3,5,11,10,11,64,10
ENRI,7,11,11,7,9,12,7,7,10,81,12


,fg%,ft%,points,rebounds,assists,steals,blocks,turnovers,threes
TANG,0.48,0.76,210.87,79.97,46.57,13.1,9.67,24.8,20.9
MAND,0.49,0.71,195.60,84.00,47.10,12.9,9.80,25.6,15.2
PATE,0.47,0.75,206.20,79.60,54.50,12.1,8.40,29.9,18.8
SWAN,0.47,0.77,215.20,68.90,45.00,12.3,8.80,24.9,25.5
GPp,0.47,0.81,231.50,68.70,39.10,11.7,8.80,24.2,22.8
VEGE,0.47,0.80,210.00,70.50,53.70,11.6,9.40,28.0,24.6
SHAR,0.47,0.80,224.60,75.40,38.70,11.6,8.50,21.9,29.0
GREW,0.47,0.82,208.40,80.90,37.50,13.1,10.40,23.0,22.0
RD,0.48,0.77,194.50,78.90,52.90,12.4,8.20,27.3,17.1
ENRI,0.47,0.74,193.40,72.00,39.70,10.2,8.70,25.3,18.8


In [58]:
trade_analyzer(rosters, "VEGE", ["Richaun Holmes"], "TANG", ["Mikal Bridges"], teamMap)

Richaun Holmes pts/reb/ast: 15.00/8.80/1.80 stl/blk/tov: 0.60/1.70/1.30/ fg%/ft%/3pm: 0.63/0.77/0.00
Mikal Bridges pts/reb/ast: 13.50/4.30/2.10 stl/blk/tov: 1.10/0.90/0.80/ fg%/ft%/3pm: 0.54/0.83/1.90
TANG beats MAND, PATE, SWAN, GPp, GREW, RD, ENRI, MA, and loses to VEGE, SHAR, SING, 

MAND beats PATE, SWAN, GPp, GREW, RD, ENRI, MA, and loses to TANG, VEGE, SHAR, SING, 

PATE beats SWAN, GPp, RD, ENRI, MA, and loses to TANG, MAND, VEGE, SHAR, GREW, SING, 

SWAN beats GPp, RD, ENRI, MA, and loses to TANG, MAND, PATE, VEGE, SHAR, GREW, SING, 

GPp beats VEGE, RD, ENRI, and loses to TANG, MAND, PATE, SWAN, SHAR, GREW, SING, MA, 

VEGE beats TANG, MAND, PATE, SWAN, GREW, RD, ENRI, SING, MA, and loses to GPp, SHAR, 

SHAR beats TANG, MAND, PATE, SWAN, GPp, VEGE, RD, ENRI, MA, and loses to GREW, SING, 

GREW beats PATE, SWAN, GPp, SHAR, RD, ENRI, MA, and loses to TANG, MAND, VEGE, SING, 

RD beats ENRI, MA, and loses to TANG, MAND, PATE, SWAN, GPp, VEGE, SHAR, GREW, SING, 

ENRI beats and l

,fg%,ft%,points,rebounds,assists,steals,blocks,turnovers,threes,Total
VEGE,9,5,6,11,1,4,1,11,4,52


,fg%,ft%,points,rebounds,assists,steals,blocks,turnovers,threes
VEGE,0.471054,0.790909,212.6,67.8,56.6,13.4,10.5,28.1,23.4


Prior to the trade TANG was ranked 4


,fg%,ft%,points,rebounds,assists,steals,blocks,turnovers,threes,Total
TANG,3,6,7,3,6,1,5,7,8,46


...

,fg%,ft%,points,rebounds,assists,steals,blocks,turnovers,threes,Total
VEGE,12,4,7,11,1,2,4,11,3,55


,fg%,ft%,points,rebounds,assists,steals,blocks,turnovers,threes
VEGE,0.465116,0.794393,211.1,63.3,56.9,13.9,9.7,27.6,25.3


After the trade TANG was ranked 1


,fg%,ft%,points,rebounds,assists,steals,blocks,turnovers,threes,Total
TANG,1,6,6,1,6,4,1,10,9,44


,fg%,ft%,points,rebounds,assists,steals,blocks,turnovers,threes
TANG,0.488559,0.779443,213.5,86.4,49.5,13.7,10.3,27.0,19.2


In [15]:
[print(p) for p in rosters[teamMap["TANG"]]]

Damian Lillard pts/reb/ast: 29.57/4.14/10.14 stl/blk/tov: 1.00/0.29/4.29/ fg%/ft%/3pm: 0.42/0.93/4.71
Devin Booker pts/reb/ast: 27.86/3.29/4.29 stl/blk/tov: 1.00/0.14/2.86/ fg%/ft%/3pm: 0.54/0.88/2.14
Paul George pts/reb/ast: 21.86/5.86/5.29 stl/blk/tov: 0.86/0.71/2.71/ fg%/ft%/3pm: 0.47/0.83/2.86
John Collins pts/reb/ast: 17.56/7.56/1.11 stl/blk/tov: 0.33/0.89/1.22/ fg%/ft%/3pm: 0.51/0.85/1.22
Christian Wood pts/reb/ast: 18.35/9.00/1.40 stl/blk/tov: 0.70/1.21/1.90/ fg%/ft%/3pm: 0.56/0.75/1.19
Brook Lopez pts/reb/ast: 10.57/5.00/0.57 stl/blk/tov: 0.57/1.29/1.29/ fg%/ft%/3pm: 0.50/0.76/1.00
Jaylen Brown pts/reb/ast: 20.43/5.14/5.57 stl/blk/tov: 0.71/0.00/2.71/ fg%/ft%/3pm: 0.43/0.88/1.86
Jaren Jackson Jr. pts/reb/ast: 0.00/0.00/0.00 stl/blk/tov: 0.00/0.00/0.00/ fg%/ft%/3pm: 0.00/0.00/0.00
Derrick White pts/reb/ast: 11.03/3.51/3.71 stl/blk/tov: 0.71/0.91/1.31/ fg%/ft%/3pm: 0.46/0.85/0.97
Nerlens Noel pts/reb/ast: 7.43/8.57/1.00 stl/blk/tov: 0.86/2.14/1.71/ fg%/ft%/3pm: 0.63/0.73/0.00
Ken

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [121]:
def getSoup(url,sleep_time ):
    driver = selenium.webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    time.sleep(sleep_time)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    return soup
def getGames(soup):
    player_table = soup.find("tbody", class_="Table2__tbody").find_all("tr", class_="Table2__tr")
    print(len(player_table))
    player_mapping = {}
    for i in range(0, len(player_table)):
        player_row = soup.find("tbody", class_="Table2__tbody").find_all("tr", class_="Table2__tr", attrs={"data-idx": str(i)})
        player_name = soup.find("tbody", class_="Table2__tbody").find("tr", class_="Table2__tr", attrs={"data-idx": str(i)}).find("td", class_= "Table2__td Table2__td--fixed-width", style = "width: 228px;").find("div")['title']
        for j in range(0,len(player_row)):
            todays_game = soup.find("tbody", class_="Table2__tbody").find("tr", class_="Table2__tr", attrs={"data-idx": str(i)}).find_all("td", class_= "Table2__td Table2__td--fixed-width", style = "width: 60px;")[1]
            todays_game = str(todays_game)
            todays_soup = BeautifulSoup(todays_game, 'html.parser')
            todays_game = todays_soup.find("div", class_="jsx-2810852873 table--cell opp ml4").getText()
#             print(todays_game)
            rest_of_games = soup.find("tbody", class_="Table2__tbody").find("tr", class_="Table2__tr", attrs={"data-idx": str(i)}).find_all("td", class_= "Table2__td", attrs={'style': None})
            game_text = ""
            for i in range(len(rest_of_games)):
                game_text = game_text + str(rest_of_games[i])
            rest_soup = BeautifulSoup(game_text, 'html.parser')
            rest_of_games = rest_soup.find_all(text=True)
            player_mapping[player_name] = [todays_game] + rest_of_games
    return player_mapping

def getAllMatchups(soup):
    soups = soup.find_all("div", class_ = "jsx-1241409305 matchup-score two-links")
    team_stats = {}
    matchups = {}
    categories = {}
    for soup in soups:
        #get team id's and construct matchup mappings
        teams = soup.find_all("li", class_="ScoreboardScoreCell__Item")
        team1_link = BeautifulSoup(str(teams[0]), 'html.parser').find("a")['href']
        team1_id = team1_link[56:58]
        if team1_id != "10":
            team1_id = team1_id[:1]
        team2_link = BeautifulSoup(str(teams[1]), 'html.parser').find("a")['href']
        team2_id = team2_link[56:58]
        if team2_id != "10":
            team2_id = team2_id[:1]
        matchups[int(team1_id)] = int(team2_id)
        matchups[int(team2_id)] = int(team1_id)
        
        
        #get stats for each team and construct player to stats mapping
        stats_table = soup.find("table", class_ = "Table2__table-scroller Table2__table")
        categories = stats_table.find("thead").find_all("th", class_ ="Table2__th")
        temp = ""
        for i in range(len(categories)):
            temp = temp + str(categories[i])
        header_soup = BeautifulSoup(temp, 'html.parser')
        categories = header_soup.find_all(text=True)
        
        player1_stats = stats_table.find("tbody").find("tr", attrs={"data-idx": str(0)}).find_all(text=True)[1:]
        player1_stats = [float(i) for i in player1_stats]
        team_stats[team1_id] = player1_stats
        
        player2_stats = stats_table.find("tbody").find("tr", attrs={"data-idx": str(1)}).find_all(text=True)[1:]
        player2_stats = [float(i) for i in player2_stats]
        team_stats[team2_id] = player2_stats
    return matchups, team_stats, categories
# def matchup_predictor(table, days, teamID, matchups, categories, team_stats):
#     schedule_url = 'https://fantasy.espn.com/basketball/team?leagueId=65116266&teamId=' + str(teamID) + '&view=schedule'    
#     soup = getSoup(schedule_url, 4)
#     player_mapping1 = getGames(soup)
#     player1_num_games = {}
#     for player in player_mapping1.keys():
#         player_mapping1[player] = player_mapping1[player][:days]
#         player1_num_games[player] = sum([1 for game in player_mapping1[player] if game != "--"])
#     schedule_url = 'https://fantasy.espn.com/basketball/team?leagueId=65116266&teamId=' + str(matchups[teamID]) + '&view=schedule'    
#     soup = getSoup(schedule_url, 4)
#     player_mapping2 = getGames(soup)
#     print(player_mapping2)
#     player2_num_games = {}
#     for player in player_mapping2.keys():
#         player_mapping2[player] = player_mapping2[player][:days]
#         player2_num_games[player] = sum([1 for game in player_mapping2[player] if game != "--"])
#     for player in player1_num_games.keys():
#         for 


In [123]:
scoreboard_url = "https://fantasy.espn.com/basketball/league/scoreboard?leagueId=1216230781"
scoreboard_soup = getSoup(scoreboard_url, 4)
matchups, team_stats, categories = getAllMatchups(scoreboard_soup)
print(matchups)
print(team_stats)
matchup_predictor(bbrTable, 1, 5, matchups, categories, team_stats)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\eric_\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


 
{}
{}


NameError: name 'matchup_predictor' is not defined